In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from PIL import Image

# Dataset MNIST - Yann LeCun

In [ ]:
# Cargar MNIST
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X = mnist.data
y = mnist.target.astype(int)

# Mostrar 10 ejemplos
plt.figure(figsize=(10, 2))
for i in range(10):
    image = X[i].reshape(28, 28)
    label = y[i]
    plt.subplot(1, 10, i + 1)
    plt.imshow(image, cmap="gray")
    plt.title(str(label))
    plt.axis("off")
plt.suptitle("Primeras 10 imágenes de MNIST")
plt.show()

print("Número de imágenes:", X.shape[0])

In [ ]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_layers, output_size, learning_rate=0.01):
        self.learning_rate = learning_rate
        self.layers = [input_size] + hidden_layers + [output_size]
        self.weights = []
        self.biases = []
        self.loss_history = []
        total_params = 0

        # Initialize weights and biases
        for i in range(len(self.layers) - 1):
            w = np.random.randn(self.layers[i], self.layers[i+1]) * np.sqrt(1. / self.layers[i])
            b = np.zeros((1, self.layers[i+1]))
            self.weights.append(w)
            self.biases.append(b)

            # Count parameters
            num_params = self.layers[i] * self.layers[i+1] + self.layers[i+1]
            print(f"  Capa {i+1}: ({self.layers[i]} x {self.layers[i+1]}) | Parámetros: {num_params}")
            total_params += num_params
        print(f"\nTotal de parámetros entrenables: {total_params}\n")

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, a):
        return a * (1 - a)

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def cross_entropy_loss(self, y_pred, y_true):
        m = y_true.shape[0]
        loss = -np.sum(y_true * np.log(y_pred + 1e-9)) / m
        return loss

    def forward(self, X):
        activations = [X]
        zs = []
        for w, b in zip(self.weights[:-1], self.biases[:-1]):
            z = np.dot(activations[-1], w) + b
            zs.append(z)
            a = self.sigmoid(z)
            activations.append(a)

        # Output layer with softmax
        z = np.dot(activations[-1], self.weights[-1]) + self.biases[-1]
        zs.append(z)
        a = self.softmax(z)
        activations.append(a)
        return activations, zs

    def backward(self, X, y, activations, zs):
        grads_w = [0] * len(self.weights)
        grads_b = [0] * len(self.biases)

        # Output layer error
        delta = activations[-1] - y
        grads_w[-1] = np.dot(activations[-2].T, delta)
        grads_b[-1] = np.sum(delta, axis=0, keepdims=True)

        # Backpropagate through hidden layers
        for l in range(len(self.layers) - 2, 0, -1):
            delta = np.dot(delta, self.weights[l].T) * self.sigmoid_derivative(activations[l])
            grads_w[l-1] = np.dot(activations[l-1].T, delta)
            grads_b[l-1] = np.sum(delta, axis=0, keepdims=True)

        return grads_w, grads_b

    def update_params(self, grads_w, grads_b, batch_size):
        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * grads_w[i] / batch_size
            self.biases[i] -= self.learning_rate * grads_b[i] / batch_size

    def train(self, X, y, epochs=100, batch_size=32):
        self.loss_history = []
        for epoch in range(epochs):
            # Shuffle data
            indices = np.arange(X.shape[0])
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]

            for i in range(0, X.shape[0], batch_size):
                X_batch = X[i:i+batch_size]
                y_batch = y[i:i+batch_size]

                activations, zs = self.forward(X_batch)
                grads_w, grads_b = self.backward(X_batch, y_batch, activations, zs)
                self.update_params(grads_w, grads_b, X_batch.shape[0])

            # Loss after epoch
            y_pred = self.forward(X)[0][-1]
            loss = self.cross_entropy_loss(y_pred, y)
            self.loss_history.append(loss)
            print(f"Epoch {epoch+1}/{epochs} - Loss: {loss:.4f}")

    def predict(self, X):
        output = self.forward(X)[0][-1]
        return np.argmax(output, axis=1)

# Preprocesamiento

In [ ]:
# Cargar MNIST
print("Cargando MNIST...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False)

In [ ]:

print(mnist.data[0])

In [ ]:
X = mnist.data / 255.0  # Normalizamos los píxeles a [0, 1]
y = mnist.target.astype(int).reshape(-1, 1)

# One-hot encode de etiquetas
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
print("Datos cargados.")

# Entrenamiendo

In [ ]:
# Inicializar la red
nn = NeuralNetwork(input_size=784, hidden_layers=[64, 32], output_size=10, learning_rate=0.1)

In [ ]:
# Entrenar
nn.train(X_train, y_train, epochs=50, batch_size=64)

plt.plot(nn.loss_history, label='Loss')
plt.xlabel("Épocas")
plt.ylabel("Pérdida (Loss)")
plt.title("Evolución del Loss durante el entrenamiento")
plt.legend()
plt.grid(True)
plt.show()

# Evaluación

In [ ]:
# Evaluar en test
predictions = nn.predict(X_test)
y_test_labels = np.argmax(y_test, axis=1)

accuracy = np.mean(predictions == y_test_labels)
print(f"Test accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Calcular matriz de confusión
cm = confusion_matrix(y_test_labels, predictions)

# Mostrar matriz de confusión
plt.figure(figsize=(8, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.arange(10))
disp.plot(cmap='Blues', values_format='d')
plt.title("Matriz de Confusión - Clasificación MNIST")
plt.grid(False)
plt.show()

In [ ]:
# Accuracy general
acc = accuracy_score(predictions, y_test_labels)
print(f"Accuracy: {acc:.4f}")

# Métricas por clase
precision = precision_score(predictions, y_test_labels, average=None)
recall = recall_score(predictions, y_test_labels, average=None)
f1 = f1_score(predictions, y_test_labels, average=None)

# Imprimir todo por clase
for i in range(10):
    print(f"Clase {i} → Precision: {precision[i]:.4f}, Recall: {recall[i]:.4f}, F1-score: {f1[i]:.4f}")

# Reporte completo (opcional)
print("\nReporte de clasificación completo:")
print(classification_report(predictions, y_test_labels))

In [ ]:
def predict_image(ruta_img, nn):
    # Cargar imagen, convertir a escala de grises y redimensionar
    img = Image.open(ruta_img).convert("L").resize((28, 28))
    img_arr = np.array(img)

    # Invertir colores si fondo blanco
    if np.mean(img_arr) > 127:
        img_arr = 255 - img_arr

    # Normalizar y aplanar
    img_norm = img_arr / 255.0
    input_data = img_norm.reshape(1, -1)

    # Predicción
    pred = nn.predict(input_data)[0]

    # Mostrar imagen y resultado
    plt.imshow(img_arr, cmap="gray")
    plt.title(f"Predicción: {pred}")
    plt.axis("off")
    plt.show()

In [ ]:
predict_image("images/example.png", nn)